<a href="https://colab.research.google.com/github/vivacitylabs/data-toolkit/blob/master/notebooks/countline_speeds_bulk_download_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Countline Speeds - Bulk Download Generator V2

## Generate a csv file of countline speed data over multiple days

This notebook only works for countlines that have countline speeds enabled.

**Note:** This notebook relies on a lot of data processing to calculate average speeds. It might crash if too much data is requested. Please be mindful when selecting number of countlines, time bucket granularity and time period.

#### How to do it

This notebook will run you through all the necessary steps and will save the output csv file locally or in your Google Drive.

You will simply need to fill in a few details and then hit the run button next to the code cells.

What you will need:

- VivaCity API login credentials
- Countline ids you want to download data for


#### Output format

You will receive data on countline speed in the following format:

Beware that direction is given as In/Out but also aggregated (Both). If numbers are added up without filtering, then volumes will be doubled.



| Countline | Start Date | Time from | Time to |Class | Direction |	total_counts |	mean_speed |	<5mph	 |	5-10mph |	10-15mph|	15-20mph | 20-25mph |25-30mph| 30-35mph | >35mph | total_over_20mph | total_over_25mph | total_over_30mph |
|:---------:|:---------:|:---------:|:---------:|:---------:|:---------:|:---------:|:---------:|:---------:|:---------:|:---------:|:---------:|:---------:|:---------:|:---------:|:---------:|:---------:|:---------:|:---------:|
| 40284| 02/03/2022 |10:00:00	| 11:00:00|	Car| Return |  10	| 24.57616361	|0 | 0|0|2	|5|	2|	1 | 0 | 0  |0| 0 |0 |






## Stage 1: Getting Started
Let's begin by importing the packages we'll need and creating some useful functions!

Hit the run button in the top left corner.

In [ ]:
#@title  { vertical-output: true, display-mode: "form" }
#@markdown **Code cell:** Run this to import functions
import requests
import getpass
import json
from datetime import date, datetime, timedelta
import pytz
import pandas as pd
import numpy as np
import csv
import time
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
from ipywidgets import interact, interactive, fixed, interact_manual, Layout, Box
import ipywidgets as widgets

def get_date_range(start_date, end_date):
    start_dates = []
    end_dates = []

    start_date = datetime.fromisoformat(start_date)
    end_date = datetime.fromisoformat(end_date)
    while True:
        start_dates.append(start_date.strftime('%Y-%m-%dT%H:%M:%S.000Z'))
        end_dates.append((start_date+timedelta(days=1)).strftime('%Y-%m-%dT%H:%M:%S.000Z'))
        start_date = start_date+timedelta(days=1)
        if start_date > end_date:
            break
    date_range = list(zip(start_dates, end_dates))
    return date_range

## Stage 2: Data Import
First, we'll input the api username and password. Contact your Customer Success Manage if you don't have these details.

We will then request all countlines the user has access to. If some countlines are missing, get in touch with us.

Finally, you will select the date period, countlines and classes to request data for.

### Authentication
Now you will need your API login details, ie. a username and a password. If you don't have one, please contact your Customer Success Manager.

1.   Enter the username into the field on the right, then hit the run button (▶).
2.   Input the password in the box that appears below it and hit "enter" on your keyboard.

In [ ]:
#@title  { run: "auto", vertical-output: true, display-mode: "form" }
#@markdown Insert your login credentials
username = "api-username" #@param {type:"string"}

auth_body = {}
auth_body['username'] = username
auth_body['password'] = getpass.getpass()

#### Available Countlines

Get access token using our username and password and get countlines user has access to.

In [ ]:
#@title { vertical-output: false, display-mode: "form" }
#@markdown **Code cell:** Run this to get authorized access to the API

print("Authorising...")
auth_response = requests.post("https://api.vivacitylabs.com/get-token", data=auth_body, headers={'Content-Type':'application/x-www-form-urlencoded'})
if auth_response.status_code == 401:
  print("\n!Error: Can't connect to the API. Check your username and password again.\nIf issues persists, ask your CSM to check with technical support if your user is setup correctly on the API\n")
else:
  headers = {}
  headers['Authorization'] = "Bearer " + auth_response.json()['access_token']
  refresh_body = {}
  refresh_body['refresh_token'] = auth_response.json()['refresh_token']
  start = time.time()
  print("Done. Successfully retrieved access token.")

In [ ]:
#@title { vertical-output: false, display-mode: "form" }
#@markdown **Code cell:** Run this to retrieve sensors and countlines available to you from the API

#get hardware meta data
print("\nRequesting metadata ...")
api_url_base = 'https://beta.api.vivacitylabs.com'
hardware_request = requests.get(f'{api_url_base}/hardware/metadata', headers=headers)
if hardware_request.status_code == 401:
  print("\n!Error: Can't access the data. Ask your CSM to check with technical support if your user is setup correctly on API 3\n")
hardware = hardware_request.json()

# Get hardware info
dict_hard = { "hardware_id" : [], "countline_id" : [], "countline_name" : [] }
for id in hardware:
  for lens in hardware[id]["view_points"]:
    for entity in hardware[id]["view_points"][lens]:
      for countline_id in hardware[id]["view_points"][lens]["countlines"]:
        dict_hard["hardware_id"].append(id)
        dict_hard["countline_id"].append(countline_id)
        dict_hard["countline_name"].append(hardware[id]["view_points"][lens]["countlines"][countline_id]['name'])

#turn into dataframe and clean up
df_hard = pd.DataFrame.from_dict(dict_hard)
df_hard["sensor_name"] = df_hard["countline_name"]
df_hard["countline_name_display"] = df_hard["countline_name"] + " (" + df_hard["countline_id"] + ")"
for i in range(len(df_hard)):
  if len(df_hard["sensor_name"].iloc[i])>1:
    df_hard["sensor_name"].iloc[i] = df_hard["sensor_name"].iloc[i][0] + "" + df_hard["sensor_name"].iloc[i][1]
  else:
    df_hard["sensor_name"].iloc[i] = df_hard["sensor_name"].iloc[i]

print(len(df_hard["countline_id"].unique()), " countlines available")

#### Select countlines and date range for querying the API


After running the code below you can select class and countlines from the dropdown. Also select the start and end dates. Ensure that the start date is before the end date.

In [ ]:
#@title  { vertical-output: true, display-mode: "form" }
#@markdown **Code cell:** Run this and then  make your selections

box_layout = Layout(display='flex', flex_flow='column',
                    align_items='stretch', border=None, width='28%')

start_date_input = widgets.DatePicker(description="Start date",layout=Layout(width='55%'))
end_date_input = widgets.DatePicker(description="End date",layout=Layout(width='55%'))
timezone = widgets.Dropdown(options=['Europe/London', "Europe/Berlin", "Australia/Sydney"],description="Timezone",layout=Layout(width='55%'))
timebucket_input = widgets.Dropdown(options=['15m', "1h", "24h"],description="Time bucket",layout=Layout(width='55%'))
class_input = widgets.SelectMultiple(
    options=[ "cyclist", "motorbike", "car", "pedestrian", "taxi", "van", "minibus", "bus", "rigid", "truck", "emergency_car", "emergency_van", "fire_engine", "escooter"],
    description='Class',  disabled=False,
    layout=Layout(width='55%', height='230px')
)
countlines_input = widgets.SelectMultiple(
    options=df_hard["countline_name_display"].sort_values().unique(),
    description='Countlines',
    disabled=False,
    layout=Layout(width='auto', height='200px')
)
items = [start_date_input, end_date_input, timezone, timebucket_input, class_input,countlines_input]
box = Box(children=items, layout=box_layout)
printmd("**Select date period and countlines**")
printmd("Hold  `Ctrl + Shift`  to select multiple classes or countlines")
print("")
box

Run the cell below to set the input parameters for the API request. Check that they look alright

In [ ]:
#@title  { vertical-output: true, display-mode: "form" }
#@markdown **Code cell:** Run this and check your selection again.
params = {}
params['countline_ids'] = df_hard[df_hard["countline_name_display"].isin(countlines_input.value)]["countline_id"].to_list()
params['classes'] = list(class_input.value)
params['time_bucket'] = timebucket_input.value

#set fixed parameters
params['speed_bucket_number'] = "98"
params['max_speed'] = "44"
params['min_speed'] = "0"
params['fill_zeros'] = "true"

#convert local datetime to UTC datetime
start_date_utc = str(pd.to_datetime(start_date_input.value).tz_localize(timezone.value).astimezone(pytz.utc))
end_date_utc = str(pd.to_datetime(end_date_input.value).tz_localize(timezone.value).astimezone(pytz.utc))

#check if dates are in correct order
if start_date_input.value > end_date_input.value:
  print("Start date is after end date, please correct your date selection")
else:
  date_range = get_date_range(start_date_utc, end_date_utc)
  printmd("**Check your selection:**\n")
  print("Dates:", start_date_input.value, "to", end_date_input.value,"\nTimebucket:", timebucket_input.value, "\nClass:", class_input.value, "\nCountlines:", countlines_input.value )

#### Getting the data

Here's the API call to get the countline speed data.

The output will tell you how many requests are made and what the progress is.

In [ ]:
#@title  {vertical-output: true, display-mode: "form" }
#@markdown **Code cell:** Run this to get data from the API

#request data
print("Requesting data ...")
df_speed = []
for i,date in enumerate(date_range):
  time_elapsed = (time.time() - start)
  if time_elapsed > 500:
    print("Reauthorising...")
    auth_response = requests.post("https://api.vivacitylabs.com/refresh-token", data=refresh_body, headers={'Content-Type':'application/x-www-form-urlencoded'})
    headers = {}
    headers['Authorization'] = "Bearer " + auth_response.json()['access_token']
    refresh_body = {}
    refresh_body['refresh_token'] = auth_response.json()['refresh_token']
    start = time.time()
    print("Done. Got new access token")
  params["from"] = date[0]
  params["to"] = date[1]
  response = requests.get("https://beta.api.vivacitylabs.com/countline/speed", params=params, headers=headers)
  response_json = response.json()

  #turn into dataframe
  df_request = {"countline_id" : [], "date":[], "timefrom": [], "timeto": [], "classes": [], "direction": [], "data":[]}
  for countline in response_json:
      for buckets in response_json[countline]:
          for _class in buckets["clockwise"].keys():
              df_request["countline_id"].append(countline)
              df_request["date"].append(pd.to_datetime(buckets["from"]).tz_convert(timezone.value).date())
              df_request["timefrom"].append(pd.to_datetime(buckets["from"]).tz_convert(timezone.value).time())
              df_request["timeto"].append(pd.to_datetime(buckets["to"]).tz_convert(timezone.value).time())
              df_request["classes"].append(_class)
              df_request["direction"].append("In")
              df_request["data"].append(buckets["clockwise"][_class])
          for _class in buckets["anti_clockwise"].keys():
              df_request["countline_id"].append(countline)
              df_request["date"].append(pd.to_datetime(buckets["from"]).tz_convert(timezone.value).date())
              df_request["timefrom"].append(pd.to_datetime(buckets["from"]).tz_convert(timezone.value).time())
              df_request["timeto"].append(pd.to_datetime(buckets["to"]).tz_convert(timezone.value).time())
              df_request["classes"].append(_class)
              df_request["direction"].append("Out")
              df_request["data"].append(buckets["anti_clockwise"][_class])
  df_request = pd.DataFrame.from_dict(df_request)
  df_speed.append(df_request)   #append all dataframes

  #print progress
  print(str(i+1) + "/" + str(len(date_range)) + ": " + str(response.status_code) + " " + response.reason)
  time.sleep(1)

print("Converting into dataframe ...")
#return single dataframe
df_speed = pd.concat(df_speed, axis=0)
df_speed = df_speed.join(pd.json_normalize(df_speed['data'])) #expand data to columns

#error when no data returned
if len(df_speed)==0:
  print("Note: No data returned for set parameters (date, countlines, classes). Please run a different request.")
else:
  print("Done")

## Stage 3: Data Processing
Now we process the raw data output and calculate the average speeds per time bucket as well as counts in speed bins.

#### General clean up

You can chose to collapse detailed classes into 8 road user groups as displayed on the dashboard. Depending on the size of your data, this step may take a while

In [ ]:
#@title  { vertical-output: false, display-mode: "form" }
#@markdown Do you want to collapse detailed classes into 8 road user types?

collapse_classes = "yes" #@param [ "yes", "no"]

print("Running script ...")

#create new column with collapsed classes and sum data
collapsed_classes = {"pedestrian":"Pedestrian","cyclist":"Cyclist","e-scooter":"Pedestrian", "motorbike":"Motorbike","car":"Car","taxi":"Car",
                     "emergency_car":"Car","van":"LGV","emergency_van":"LGV",
                     "bus":"Bus","minibus":"Bus","rigid":"OGV1","fire_engine":"OGV1","truck":"OGV2",}
if collapse_classes == "yes":
  df_speed["class"] = df_speed["classes"].map(collapsed_classes)
  df_speed = df_speed.drop(columns=["data", "classes"]).groupby(['countline_id', 'date', 'timefrom', 'timeto', 'class', 'direction']).sum().reset_index()
else:
  df_speed["class"] = df_speed["classes"]

#clean up direction
df_speed_dir = df_speed.drop(columns=["direction"]).groupby(['countline_id','date', 'timefrom', 'timeto', 'class']).sum().reset_index() #sum per direction
df_speed_dir["direction"] =  "Both"

#add combined direction to dataframe
df_speed = pd.concat([df_speed,df_speed_dir], axis=0).sort_values(by=['countline_id', 'date', 'timefrom', 'timeto', 'class']).reset_index(drop=True)
print("Done.")

#### Calculate average speed

Depending on the size of your data, this may take a while.

In [ ]:
#@title  { vertical-output: false, display-mode: "form" }
#@markdown **Code cell:** Run this to calculate average speed

#recreate data in dictionary for calculating means
print("Preparing data... Time:", datetime.now())
df_speed["data"] = [ dict(zip(df_speed.columns[7:], df_speed[df_speed.columns[7:]].iloc[i].to_list())) for i in df_speed.index]
print("Done. Prepared data. Time:", datetime.now())

#reate mean using data stored in dictionary
print("\nCalculating mean ...")
def mean_from_histo(histogram):
    total_speed = 0
    total_counts = sum(histogram.values())

    #calculate average speed
    for key in histogram:
        total_speed += float(key)*histogram[key]
    if total_counts != 0:
        avg_speed = total_speed / total_counts
    else:
        avg_speed = np.nan
    return round(avg_speed,2)

#append data
total_counts = []
mean = []

for row in df_speed["data"]:
    total_counts.append(sum(row.values()))
    mean.append(mean_from_histo(row))

df_speed["total_counts"] = total_counts
df_speed["mean"] = mean
df_speed["mean"] = df_speed["mean"] * 2.236936
print("Done. Calculated mean")

#### Calculate counts in speed bins

In [ ]:
#@title  { vertical-output: false, display-mode: "form" }
#@markdown **Code cell:** Run this to group counts into 5mph speed bins
print("Creating speed bins ...")
speed_bins = {
          "<5mph" : ["0.00","0.45","0.90","1.35","1.80"],
          "5-10mph": ["2.24","2.69","3.14","3.59","4.04"],
          "10-15mph":["4.49","4.94","5.39","5.84","6.29"],
          "15-20mph" : ["6.73","7.18","7.63","8.08","8.53"],
          "20-25mph": ["8.98","9.43","9.88","10.33","10.78"],
          "25-30mph": ["11.22","11.67","12.12","12.57","13.02"],
          "30-35mph": ["13.47","13.92","14.37","14.82","15.27"],
          ">35mph": ["15.71","16.16","16.61","17.06","17.51","17.96","18.41","18.86","19.31","19.76", "20.20","20.65","21.10","21.55","22.00",
                     "22.45","22.90","23.35","23.80","24.24", "24.69","25.14","25.59","26.04","26.49", "26.94","27.39","27.84","28.29","28.73",
                     "29.18","29.63","30.08","30.53","30.98", "31.43","31.88","32.33","32.78","33.22", "33.67","34.12","34.57","35.02","35.47",
                     "35.92","36.37","36.82","37.27","37.71", "38.16","38.61","39.06","39.51","39.96", "40.41","40.86","41.31","41.76","42.20","42.65","43.10","43.55"]
               }

for bin in speed_bins:
  df_speed[bin] = df_speed[speed_bins[bin]].sum(axis=1)

print("Done. Created speed bins.")

#### Clean up file before export

In [ ]:
#@title  { vertical-output: false, display-mode: "form" }
#@markdown **Code cell:** Run this to clean up data before exporting
print("Cleaning up ...")

#clean up dataframe
df_speed_final = df_speed[['countline_id', 'date','timefrom','timeto','class','direction','total_counts','mean','<5mph','5-10mph','10-15mph','15-20mph',
                                 '20-25mph','25-30mph','30-35mph', '>35mph']]

#rename columns
column_names = {'countline_id': "countline_id",  'date': "date",'timefrom': "time_from",'timeto': "time_to",'class': "class",'direction': "direction",
                'total_counts': "total_counts",'mean': "mean_speed",'<5mph': '<5mph','5-10mph': '5-10mph','10-15mph': '10-15mph','15-20mph': '15-20mph',
                                 '20-25mph': '20-25mph','25-30mph': '25-30mph','30-35mph': '30-35mph', '>35mph': '>35mph'}
df_speed_final = df_speed_final.rename(columns=column_names)

#add more aggregations
df_speed_final["total_over_20mph"] = df_speed_final[[ '20-25mph','25-30mph','30-35mph', '>35mph' ]].sum(axis=1)
df_speed_final["total_over_25mph"] = df_speed_final[['25-30mph','30-35mph', '>35mph' ]].sum(axis=1)
df_speed_final["total_over_30mph"] = df_speed_final[['30-35mph', '>35mph' ]].sum(axis=1)

print("Done. File ready for export.")

## Stage 4: Data Export
Now let's write this to a .csv file. You can either save the file locally (it will show in your Downloads folder) or save it to a Google Drive.


* **Local Downloads Folder:** This might not work if your browser or computer blocking downloads.
* **Google Drive:** If you want to save it in Google Drive, you will be asked for permission to connect to your Google Account.

In [ ]:
#@title  { vertical-output: true, display-mode: "form" }
#@markdown Select where to save the csv file
download_location = "Local folder" #@param [ "Local folder", "Google Drive"]
#@markdown Name your file
filename = "countline-speeds-test" #@param {type:"string"}
#@markdown Hit run (>)

if download_location == "Local folder":
  from google.colab import files
  df_speed_final.to_csv(filename + ".csv", index = False)
  files.download(filename + ".csv")
else:
  from google.colab import drive
  drive.mount('/content/drive')
  path = '/content/drive/My Drive/'
  df_speed_final.to_csv(path + filename +".csv", index = False)